In [ ]:
"""
Purpose: To Create the table that
will store the neuron objects that have finer
axon preprocessing

"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-28 02:20:34,130 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 02:20:34,132 - settings - Setting database.user to celiib
INFO - 2021-05-28 02:20:34,133 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 02:20:34,137 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-28 02:20:34,138 - settings - Setting enable_python_native_blobs to True


Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-28 02:20:42,190 - connection - Connected celiib@at-database.ad.bcm.edu:3306
INFO - 2021-05-28 02:20:42,650 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-28 02:20:42,655 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 02:20:42,656 - settings - Setting database.user to celiib
INFO - 2021-05-28 02:20:42,657 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 02:20:42,658 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 02:20:42,658 - settings - Setting database.user to celiib
INFO - 2021-05-28 02:20:42,659 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 02:20:42,662 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-28 02:20:43,146 - settings - Setting enable_python_native_blobs to True


In [2]:
#so that it will have the adapter defined
from datajoint_utils import *

In [3]:
test_mode = True

# Debugging the contains method

In [4]:
import system_utils as su

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-05-28 02:20:43,265 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 02:20:43,266 - settings - Setting database.user to celiib
INFO - 2021-05-28 02:20:43,268 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 02:20:43,269 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 02:20:43,270 - settings - Setting database.user to celiib
INFO - 2021-05-28 02:20:43,270 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 02:20:43,273 - settings - Setting enable_python_native_blobs to True


Sleeping 141 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-05-28 02:20:43,829 - settings - Setting enable_python_native_blobs to True


In [6]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

import datajoint as dj

from pykdtree.kdtree import KDTree
import trimesh_utils as tu
import proofreading_utils as pru
import numpy as np

# Defining the Table

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 3948


# Proofreading Version

In [8]:
@schema
class DecompositonAxonVersion(dj.Manual):
    definition="""
    axon_version      : tinyint unsigned  # key by which to lookup the finer axon processing method
    ---
    description          : varchar(256)    # new parts of the finer axon preprocessing
    """
versions=[[0,"axon with standard meshparty"],
          [2,"axon with finer resolution"],
         [4,"even more fine resoution, axon skeleton, boutons, webbing"],
         [5,"filtered away floating pieces near soma for stitching"]]

dict_to_write = [dict(axon_version=k,description=v) for k,v in versions]
DecompositonAxonVersion.insert(dict_to_write,skip_duplicates=True)

DecompositonAxonVersion()

axon_version key by which to lookup the finer axon processing method,description new parts of the finer axon preprocessing
0,axon with standard meshparty
2,axon with finer resolution
4,"even more fine resoution, axon skeleton, boutons, webbing"
5,filtered away floating pieces near soma for stitching


In [10]:
# minnie,schema = du.configure_minnie_vm()
# minnie.DecompositionAxon.delete()
#minnie.schema.external['decomposition'].delete(delete_external_files=True)

In [11]:
import numpy as np
import time
import classification_utils as clu
import proofreading_utils as pru
import axon_utils as au

axon_version = au.axon_version

verbose = True

@schema
class DecompositionAxon(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    -> minnie.DecompositonAxonVersion()             # the version of code used for this cell typing classification
    ---
    decomposition        : <decomposition> # saved neuron object with high fidelity axon
    axon_length: double  # length (in um) of the classified axon skeleton
    run_time=NULL : double                   # the amount of time to run (seconds)
    """
                             
    
    #key_source = minnie.Decomposition() & minnie.NucleiSegmentsRun2() & "segment_id=864691136540183458"
    key_source = (minnie.Decomposition() & 
                  du.current_validation_segment_id_restriction
                  - du.current_validation_segment_id_exclude)
    
    

    def make(self,key):
        """
        Pseudocode:
        1) Pull Down all the Neuron Objects associated with a segment_id
        
        For each neuron:
        2) Run the full axon preprocessing
        3) Save off the neuron
        4) Save dict entry to list
        
        
        5) Write the new entry to the table

        """
        
        
        # 1) Pull Down All of the Neurons
        segment_id = key["segment_id"]
        
        if verbose:
            print(f"------- Working on Neuron {segment_id} -----")
        
        whole_pass_time = time.time()
        
        #1) Pull Down all the Neuron Objects associated with a segment_id
        neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)

        if verbose:
            print(f"Number of Neurons found ={len(neuron_objs)}")

        #For each neuron:
        dict_to_write = []
        for split_index,neuron_obj in zip(neuron_split_idxs,neuron_objs):
            
            if verbose:
                print(f"--> Working on Split Index {split_index} -----")
                
            st = time.time()
            #Run the Axon Decomposition
            neuron_obj_with_web = au.complete_axon_processing(neuron_obj,
                                     verbose=True)
            
            save_time = time.time()
            ret_file_path = neuron_obj_with_web.save_compressed_neuron(
                                            output_folder=str(du.get_decomposition_path()),
                                            #output_folder = "./",
            file_name=f"{neuron_obj_with_web.segment_id}_{split_index}_split_axon_v{au.axon_version}",
                                              return_file_path=True,
                                             export_mesh=False,
                                             suppress_output=True)

            ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
            
            if verbose:
                print(f"ret_file_path_str = {ret_file_path_str}")
                print(f"Save time = {time.time() - save_time}")
            
            n_dict = dict(key,
              split_index = split_index,
              axon_version = au.axon_version,
             decomposition=ret_file_path_str,
             axon_length=neuron_obj_with_web.axon_length,
              run_time = np.round(time.time() - st,2)
             )
            
            dict_to_write.append(n_dict)
        
        #write the
        self.insert(dict_to_write,skip_duplicates=True,allow_direct_insert=True)

        print(f"\n\n ***------ Total time for {key['segment_id']} = {time.time() - whole_pass_time} ------ ***")

# Running the Populate

In [12]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_axon'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "status='error'") #& "error_message='IndexError: list index out of range'"

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition_axon,3a9977fe0309921114d7ad948205d19d,error,=BLOB=,IndexError: arrays used as indices must be of integer (or boolean) type,=BLOB=,celiib@10.28.0.133,at-node6,1,455530,2021-05-24 17:11:59
__decomposition_axon,48a5aef7c4abd7fced95d3dbeed140ed,error,=BLOB=,IndexError: arrays used as indices must be of integer (or boolean) type,=BLOB=,celiib@10.28.0.133,at-node6,1,455530,2021-05-24 17:13:12
__decomposition_axon,88cc23cf175d74be27a56afd0262035c,error,=BLOB=,"Exception: Segment ID 864691136672696199 should have been in DecompositionSplit but wasnt with n_error_limbs = 4, n_somas = 2",=BLOB=,celiib@10.28.0.153,at-node26,1,455531,2021-05-24 16:48:21


In [13]:
import time
pru = reload(pru)
nru = reload(nru)
import neuron_searching as ns
ns = reload(ns)
clu = reload(clu)
du = reload(du)
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    DecompositionAxon.populate(reserve_jobs=True, suppress_errors=True, order="random")
else:
    DecompositionAxon.populate(reserve_jobs=True, suppress_errors=False,)# order="random")
print('Populate Done')

print(f"Total time for DecompositionAxon populate = {time.time() - start_time}")


INFO - 2021-05-28 02:22:19,935 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 02:22:19,937 - settings - Setting database.user to celiib
INFO - 2021-05-28 02:22:19,938 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 02:22:19,939 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 02:22:19,940 - settings - Setting database.user to celiib
INFO - 2021-05-28 02:22:19,940 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 02:22:19,945 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-28 02:22:20,561 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-28 02:22:20,840 - autopopulate - Found 130 keys to populate
INFO - 2021-05-28 02:22:20,856 - connection - Transaction started
INFO - 2021-05-28 02:22:20,859 - autopopulate - Populating: {'segment_id': 864691135585319036, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': Decimal('88.0

Populate Started
------- Working on Neuron 864691135585319036 -----
**Using table __decomposition_split for table_to_neuron_objs**


INFO - 2021-05-28 02:22:24,564 - connection - Transaction cancelled. Rolling back ...
ERROR - 2021-05-28 02:22:24,775 - ultratb - Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO - 2021-05-28 02:22:24,783 - ultratb - 
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-e475d8179e83>", line 17, in <module>
    DecompositionAxon.populate(reserve_jobs=True, suppress_errors=False,)# order="random")
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "<ipython-input-11-2701215e6c73>", line 56, in make
    neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)
  File "/meshAfterParty/meshAfterParty/datajoint_utils.py", line 1544, in decomposition_with_spine_recalculation
    return_split_index=True)
  File "/meshAfterParty/meshAfterParty/datajoint_utils.py", line 1281, in table_to_neuron_objs
    decomp_list = [filepath_to_neuron_obj(k) for k in decomp_filepaths]
  File "/meshAfterParty/meshAfterParty/datajoint_utils.py", line 1281, in <li

TypeError: object of type 'NoneType' has no len()